In [2]:
import os
from dotenv import load_dotenv
load_dotenv('../.env')  # adjust the path as needed
print(os.environ.get('TORCH_CUDA_ARCH_LIST', ''));  # to silence linter warnings

12.1


In [10]:
!~/.local/bin/uv pip show torch

Using Python 3.12.3 environment at: /home/admin/_github/massimodipaolo/ai-crash-course/.venv
Name: torch
Version: 2.9.0+cu130
Location: /home/admin/_github/massimodipaolo/ai-crash-course/.venv/lib/python3.12/site-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas, nvidia-cuda-cupti, nvidia-cuda-nvrtc, nvidia-cuda-runtime, nvidia-cudnn-cu13, nvidia-cufft, nvidia-cufile, nvidia-curand, nvidia-cusolver, nvidia-cusparse, nvidia-cusparselt-cu13, nvidia-nccl-cu13, nvidia-nvjitlink, nvidia-nvshmem-cu13, nvidia-nvtx, setuptools, sympy, triton, typing-extensions
Required-by: sentence-transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
# mask language model: predict masked token given surrounding tokens
from transformers import pipeline
import torch
model = "google-bert/bert-base-cased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
fill_mask = pipeline("fill-mask", model=model, device=device)
#print(fill_mask.model.config)
result = fill_mask("The [MASK] of Italy is Rome.")
display(result)

/home/admin/_github/massimodipaolo/ai-crash-course/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


Some weights of the model checkpoint at google-bert/bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda
/home/admin/_github/massimodipaolo/ai-crash-course/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) -

[{'score': 0.9872570037841797,
  'token': 2364,
  'token_str': 'capital',
  'sequence': 'The capital of Italy is Rome.'},
 {'score': 0.00822779256850481,
  'token': 6299,
  'token_str': 'Capital',
  'sequence': 'The Capital of Italy is Rome.'},
 {'score': 0.0007266400498338044,
  'token': 2642,
  'token_str': 'centre',
  'sequence': 'The centre of Italy is Rome.'},
 {'score': 0.0007245848537422717,
  'token': 2057,
  'token_str': 'center',
  'sequence': 'The center of Italy is Rome.'},
 {'score': 0.0004981357487849891,
  'token': 15979,
  'token_str': 'birthplace',
  'sequence': 'The birthplace of Italy is Rome.'}]

In [2]:
# but also, with high score...
result = fill_mask("The [MASK] of Spain is Rome.")
display(result)

# because during pre-training it learned: The [MASK] of <country> is <city>
# saw thousands of sentences like "The capital of France is Paris."
# or more generic patterns like...
result = fill_mask("The [MASK] of _____ is _____.")
display(result)

# no fact-checking mechanism! no knowledge awareness!
# it's purely probabilistic based on co-occurrence patterns in training data

[{'score': 0.9341114163398743,
  'token': 2364,
  'token_str': 'capital',
  'sequence': 'The capital of Spain is Rome.'},
 {'score': 0.014675662852823734,
  'token': 6299,
  'token_str': 'Capital',
  'sequence': 'The Capital of Spain is Rome.'},
 {'score': 0.010236961767077446,
  'token': 1946,
  'token_str': 'seat',
  'sequence': 'The seat of Spain is Rome.'},
 {'score': 0.002829367760568857,
  'token': 10063,
  'token_str': 'patron',
  'sequence': 'The patron of Spain is Rome.'},
 {'score': 0.0026473377365618944,
  'token': 2057,
  'token_str': 'center',
  'sequence': 'The center of Spain is Rome.'}]

[{'score': 0.22043129801750183,
  'token': 1271,
  'token_str': 'name',
  'sequence': 'The name of _ _ _ _ _ is _ _ _ _ _.'},
 {'score': 0.03419250249862671,
  'token': 1641,
  'token_str': 'title',
  'sequence': 'The title of _ _ _ _ _ is _ _ _ _ _.'},
 {'score': 0.030494004487991333,
  'token': 4134,
  'token_str': 'address',
  'sequence': 'The address of _ _ _ _ _ is _ _ _ _ _.'},
 {'score': 0.026066243648529053,
  'token': 2351,
  'token_str': 'author',
  'sequence': 'The author of _ _ _ _ _ is _ _ _ _ _.'},
 {'score': 0.023992838338017464,
  'token': 5754,
  'token_str': 'definition',
  'sequence': 'The definition of _ _ _ _ _ is _ _ _ _ _.'}]

In [3]:
# are llm training data biased?
# datasets used: 
# https://huggingface.co/datasets/legacy-datasets/wikipedia
# https://huggingface.co/datasets/bookcorpus/bookcorpus
prompts = [
        "The black man worked as a [MASK].",
        "The woman worked as a [MASK].",
        "The airplane pilot was a [MASK].",
        "The secretary was a [MASK]."
]
results = []
for i, prompt in enumerate(prompts):
    result = fill_mask(prompt)
    results.append({ "prompt": prompt, "result": result[:3] })

display(results)

[{'prompt': 'The black man worked as a [MASK].',
  'result': [{'score': 0.07436051219701767,
    'token': 17989,
    'token_str': 'waiter',
    'sequence': 'The black man worked as a waiter.'},
   {'score': 0.044776901602745056,
    'token': 18343,
    'token_str': 'bartender',
    'sequence': 'The black man worked as a bartender.'},
   {'score': 0.037703704088926315,
    'token': 9140,
    'token_str': 'detective',
    'sequence': 'The black man worked as a detective.'}]},
 {'prompt': 'The woman worked as a [MASK].',
  'result': [{'score': 0.16927538812160492,
    'token': 7439,
    'token_str': 'nurse',
    'sequence': 'The woman worked as a nurse.'},
   {'score': 0.15011048316955566,
    'token': 15098,
    'token_str': 'waitress',
    'sequence': 'The woman worked as a waitress.'},
   {'score': 0.056002069264650345,
    'token': 13487,
    'token_str': 'maid',
    'sequence': 'The woman worked as a maid.'}]},
 {'prompt': 'The airplane pilot was a [MASK].',
  'result': [{'score': 0.

In [4]:
from transformers import pipeline
# can predict?
prompt = "In a distant future, humanity has"
text_generator = pipeline("text-generation", model=model)
result = text_generator(prompt, max_new_tokens=25, truncation=True, num_return_sequences=1)
print("\n")
print(result[0]['generated_text'])

print("\n---\n")
# Q&A?
print(text_generator("who are you?", max_new_tokens=25, truncation=True, num_return_sequences=1)[0]['generated_text'])

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Device set to use cuda:0




In a distant future, humanity has.........................

---

who are you?.........................


In [6]:
#causal language model: predict next token given previous tokens
from transformers import pipeline
prompt = "In a distant future, humanity has"
text_generator = pipeline("text-generation", model="Qwen/Qwen3-0.6B")
result = text_generator(prompt, max_new_tokens=25, truncation=True, num_return_sequences=1)
print("\n")
print(result[0]['generated_text'])

Device set to use cuda:0




In a distant future, humanity has developed a method to simulate the Earth's surface using a computer program. This program uses a coordinate system where each point is represented
